In [1]:
from jupyter_client import find_connection_file
connection_file = find_connection_file()
print(connection_file)

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-17295RCP41eZxzLQg.json


In [2]:
# Constants for this project
import constants
from constants import *

import auxFunctions as af

import importlib

import os
import numpy as np
import pandas as pd

import papermill as pm
import shutil
shutil.copy('constants.py', 'export/')
shutil.copy('auxFunctions.py', 'export/')

# Change the current working directory
os.chdir("/home/yonatan/Documents/projects/vaccines/code")

In [3]:
print(dataset_day_dicts_for_adjFC)

[{'Dataset': 'GSE125921.SDY1529', 'Days': ['D0', 'D84']}, {'Dataset': 'GSE13699.SDY1289', 'Days': ['D0', 'D28']}, {'Dataset': 'GSE169159', 'Days': ['D0', 'D42']}, {'Dataset': 'GSE41080.SDY212', 'Days': ['HAI.D0', 'HAI.D28']}, {'Dataset': 'GSE45735.SDY224', 'Days': ['HAI.D21', 'HAI.D0']}, {'Dataset': 'GSE47353.SDY80', 'Days': ['D70.nAb', 'D0.nAb']}, {'Dataset': 'GSE48018.SDY1276', 'Days': ['nAb.D28', 'nAb.D0']}, {'Dataset': 'GSE48023.SDY1276', 'Days': ['HAI.D14', 'HAI.D0']}, {'Dataset': 'GSE52245.SDY1260', 'Days': ['D0', 'D30']}, {'Dataset': 'GSE59635.SDY63', 'Days': ['HAI.D0', 'HAI.D28']}, {'Dataset': 'GSE59654.SDY180', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59654.SDY404', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59654.SDY520', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59743.SDY400', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE79396.SDY984', 'Days': ['D0', 'D28']}, {'Dataset': 'SDY1325', 'Days': ['D0', 'D28']}, {'Dataset': 'SDY296', 'Days': ['D0.HAI', 'D28.HAI

In [4]:
# Read in Data and drop missing values
importlib.reload(af)
data_dir = af.get_dir_by_name("data")
df = pd.read_csv(os.path.join(data_dir, "../data/all_vaccines.csv"))
datasets = df.dropna(subset=[immage_col, age_col, dataset_col, uid_col, day_col, response_col])
dataset_names = datasets[dataset_col].unique()
# Get the info for all influenza datasets, excluding some.
influenza_df = pd.DataFrame(influenza_dicts)
all_sets_df = pd.DataFrame(dataset_day_dicts_for_adjFC)

# We can choose to analyze all datasets, only influenza ones and only non-influenza ones
if bInfluenza:
    # Get the info for all influenza datasets, excluding some.
    metadata = influenza_df
    dataset_names = metadata[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))
elif bNonInfluenza:
    metadata = all_sets_df
    dataset_names = all_sets_df[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(influenza_df[dataset_col]) - set(exclude_datasets))
else:
    metadata = all_sets_df
    dataset_names = all_sets_df[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))


# dataset_names = ["GSE48023.SDY1276"]
# Filter datasets and metadata according to the list of datasets we want to look at.
datasets = datasets.loc[datasets["Dataset"].isin(dataset_names)]
metadata =  metadata.loc[metadata["Dataset"].isin(dataset_names)]

age_str = f", subjects over the age of {age_threshlod}" if bOlderOnly else ""
print(f"""Analysis parameters: discarding serprotected subjects: {bDiscardSeroprotected}{age_str}""")

# Turn on debug here for running a single dataset through analyze_dataset()
bDebug = False
if bDebug:
    # Narrow to a specific datset
    dataset_name = "GSE48023.SDY1276"
    # Filter data
    name_mask = datasets[dataset_col] == dataset_name
    dataset = datasets.loc[name_mask].reset_index(drop=True)

    # Filter metadata
    name_mask = metadata[dataset_col] == dataset_name
    metadata = metadata.loc[name_mask].reset_index(drop=True)

    day = [x for x in metadata["Days"].iloc[0] if "D0" not in x][0]
    day0 = [x for x in metadata["Days"].iloc[0] if "D0" in x][0]

    strains = dataset.loc[dataset[day_col] == day][strain_col].unique()
    print(strains)
    if len(strains) > 1:
        # "Influenza" denotes an MFC calculation in the original dataset
        strains = list(set(strains) - set(["Influenza"])) 
        # Sort to maintain consistency (sorts in-place)
        strains.sort() 
    if len(strains) < 1:
        strains = ['single strain - missing data']

    strain_index = 0
    P = {
        "bAdjustMFC": bAdjustMFC,
        "dataset_name": dataset_name,
        "strain_index": strain_index,
        "day":  day,
        "day0":  day0,
        "strain": strains[strain_index],
        "strains": strains,
        "bPlotOnly": False,
        "bPlotThreshold": False,
    }
    af.analyze_dataset(dataset, P)

else:
    results = af.analyze_all_datasets(datasets, metadata)

Analysis parameters: discarding serprotected subjects: True
GSE125921.SDY1529
['Yellow Fever']
Preprocessing dataset, computing adjusted FC
GSE13699.SDY1289
['Yellow Fever']
Preprocessing dataset, computing adjusted FC


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


               uid  IMMAGE   Age   D0     D28  FC  Cluster  adjFC
0   SUB187451.1289    0.08 26.00 0.00  320.00 inf        0    NaN
1   SUB187452.1289   -0.18 23.00 0.00  640.00 inf        0    NaN
2   SUB187453.1289   -0.06 28.00 0.00  640.00 inf        0    NaN
3   SUB187454.1289    0.15 28.00 0.00 1280.00 inf        0    NaN
4   SUB187455.1289   -0.25 25.00 0.00  640.00 inf        0    NaN
5   SUB187456.1289    0.28 43.00 0.00  160.00 inf        0    NaN
6   SUB187457.1289   -0.20 31.00 0.00  640.00 inf        0    NaN
7   SUB187458.1289   -0.28 22.00 0.00  160.00 inf        0    NaN
8   SUB187460.1289    0.07 26.00 0.00  320.00 inf        0    NaN
9   SUB187461.1289   -0.17 23.00 0.00 1280.00 inf        0    NaN
10  SUB187462.1289   -0.01 34.00 0.00  640.00 inf        0    NaN
11  SUB187463.1289    0.00 27.00 0.00 2560.00 inf        0    NaN
12  SUB187464.1289    0.09 25.00 0.00 1280.00 inf        0    NaN
13  SUB187465.1289   -0.02 29.00 0.00  640.00 inf        0    NaN
GSE169159


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Preprocessing dataset, computing adjusted FC
Preprocessing dataset, computing adjusted FC


...

/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' 

Preprocessing dataset, computing adjusted FC
Preprocessing dataset, computing adjusted FC
Preprocessing dataset, computing adjusted FC


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


SDY67
['A/California/7/2009', 'A/Perth/16/2009']
Preprocessing dataset, computing adjusted FC
Preprocessing dataset, computing adjusted FC


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [5]:
results

,Dataset,Strain,strain_index,Day,bAdjustMFC,"(F score, IMMAGE)","(F score, Age)","(F score, Multivariate)","(NR rate over threshold, IMMAGE)","(NR rate over threshold, Age)","(NR rate over threshold, Multivariate)","(NR rate under threshold, IMMAGE)","(NR rate under threshold, Age)","(NR rate under threshold, Multivariate)","(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)"
0,GSE13699.SDY1289,Yellow_Fever,0,D28,False,0.88,0.83,0.85,0.50,0.76,0.85,0.85,NaN,0.50,0.69,0.80,0.85
1,GSE169159,WT_SARS-CoV-2,0,AdjMFC,True,0.60,0.47,0.50,0.26,0.43,0.75,0.75,0.24,0.26,0.43,0.45,0.63
2,GSE169159,WT_SARS-CoV-2,0,D42,False,0.60,0.47,0.50,0.23,0.43,0.75,0.80,0.24,0.26,0.42,0.45,0.63
3,GSE41080.SDY212,A_Brisbane_10_2007,0,AdjMFC,True,0.47,0.53,0.55,0.38,0.71,0.70,0.00,0.28,0.25,0.42,0.62,0.63
4,GSE41080.SDY212,A_Brisbane_10_2007,0,HAI.D28,False,0.69,0.70,0.74,0.63,0.68,0.77,0.16,0.28,0.25,0.66,0.69,0.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,SDY296,B_Brisbane_60_2008,2,D28.HAI,False,0.70,0.68,0.66,0.61,0.00,0.78,0.00,0.58,0.44,0.65,0.34,0.72
79,SDY67,A_California_7_2009,0,AdjMFC,True,0.58,0.64,0.67,0.00,0.59,0.60,0.48,0.25,0.20,0.29,0.62,0.63
80,SDY67,A_California_7_2009,0,HAI.D28,False,0.63,0.73,0.73,0.53,0.69,0.68,NaN,0.25,0.21,0.58,0.71,0.70
81,SDY67,A_Perth_16_2009,1,AdjMFC,True,0.46,0.45,0.43,0.38,0.37,0.37,0.17,0.19,0.24,0.42,0.41,0.40


In [6]:
# Get all the analyses that look somewhat promising based on the composite (F1 and over-threshold rate) metric
score_mask = (results["Composite", "IMMAGE"] >  results["Composite", "Age"]) | (results["Composite", "Multivariate"] >  results["Composite", "Age"])
cols_to_access = [
    dataset_col,
    strain_col,
    strain_index_col,
    day_col,
    "max_difference"
]

# Dynamically add all sub-columns for 'Composite'
composite_columns = [col for col in results.columns if col[0] == 'Composite']
cols_to_access.extend(composite_columns)
results["max_difference"] = results.apply(lambda row: max(row["Composite", "IMMAGE"] - row["Composite", "Age"], row["Composite", "Multivariate"] - row["Composite", "Age"]), axis=1)
results = results.loc[score_mask]
results = results.sort_values(by="max_difference", ascending=False)
results = results.loc[results["max_difference"] > 0.1]

In [7]:
results

,Dataset,Strain,strain_index,Day,bAdjustMFC,"(F score, IMMAGE)","(F score, Age)","(F score, Multivariate)","(NR rate over threshold, IMMAGE)","(NR rate over threshold, Age)","(NR rate over threshold, Multivariate)","(NR rate under threshold, IMMAGE)","(NR rate under threshold, Age)","(NR rate under threshold, Multivariate)","(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)",max_difference
78,SDY296,B_Brisbane_60_2008,2,D28.HAI,False,0.70,0.68,0.66,0.61,0.00,0.78,0.00,0.58,0.44,0.65,0.34,0.72,0.38
43,GSE59654.SDY180,A_Brisbane_10_2007,0,AdjMFC,True,0.69,0.60,0.60,0.25,0.29,1.00,0.50,1.00,0.29,0.47,0.44,0.80,0.36
44,GSE59654.SDY180,A_Brisbane_10_2007,0,HAI.D28,False,0.69,0.67,0.67,0.33,0.30,1.00,0.67,1.00,0.30,0.51,0.48,0.83,0.35
67,GSE59743.SDY400,B_Wisonsin_01_2010,2,AdjMFC,True,0.86,0.64,0.79,1.00,0.58,1.00,0.18,0.18,0.22,0.93,0.61,0.90,0.32
52,GSE59654.SDY404,A_Perth_16_2009,1,HAI.D28,False,0.57,0.57,0.66,0.33,0.50,1.00,0.55,0.25,0.30,0.45,0.53,0.83,0.30
32,GSE52245.SDY1260,IgG_serotype_C,5,D30,False,0.62,0.45,0.57,0.56,0.22,0.67,0.19,0.42,0.21,0.59,0.34,0.62,0.28
29,GSE52245.SDY1260,IgG_serotype_A,4,AdjMFC,True,0.49,0.53,0.63,0.15,0.38,0.80,0.41,0.21,0.20,0.32,0.45,0.72,0.26
30,GSE52245.SDY1260,IgG_serotype_A,4,D30,False,0.53,0.52,0.53,0.23,0.24,0.75,0.75,0.60,0.23,0.38,0.38,0.64,0.26
7,GSE41080.SDY212,B_Florida_4_2006,2,AdjMFC,True,0.63,0.56,0.58,0.90,0.47,0.49,0.27,0.09,0.08,0.76,0.52,0.53,0.25
69,SDY1325,Neisseria_meningitidis_strain_A_(F8238),0,AdjMFC,True,0.73,0.65,0.65,0.67,0.25,0.57,0.17,0.57,0.25,0.70,0.45,0.61,0.25


In [8]:
# Generate figures for promising datasets
for index, row in results.iterrows():
    parameters = {
        "bAdjustMFC" : row["bAdjustMFC"],
        "dataset_name": row[dataset_col],
        "strain_index": row[strain_index_col],
        "day": row[day_col]
    }
    print(f'exporting {row[dataset_col]}, strain no. {row[strain_index_col]}: {row[strain_col]}, day: {row[day_col]}')
    output_notebook_name = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}"
    output_notebook = f"export/{output_notebook_name}.ipynb"
    try:
            pm.execute_notebook(
                    input_path="vaccines-4.ipynb",
                    output_path=output_notebook,
                    parameters=parameters,
                    prepare_only=True
            )
    except Exception as e:
            print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
            raise(e)
    # Export the executed notebook to HTML
    output_html = f"{output_notebook_name}.html"
    os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

exporting SDY296, strain no. 2: B_Brisbane_60_2008, day: D28.HAI


[NbConvertApp] Converting notebook export/SDY296_B_Brisbane_60_2008_D28.HAI_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 543006 bytes to export/SDY296_B_Brisbane_60_2008_D28.HAI_discard_seroprotected.html


exporting GSE59654.SDY180, strain no. 0: A_Brisbane_10_2007, day: AdjMFC


[NbConvertApp] Converting notebook export/GSE59654.SDY180_A_Brisbane_10_2007_AdjMFC_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 492212 bytes to export/GSE59654.SDY180_A_Brisbane_10_2007_AdjMFC_discard_seroprotected.html


exporting GSE59654.SDY180, strain no. 0: A_Brisbane_10_2007, day: HAI.D28


...

[NbConvertApp] Converting notebook export/GSE52245.SDY1260_IgG1_serotype_C_AdjMFC_discard_seroprotected.ipynb to html
Traceback (most recent call last):
  File "/home/yonatan/miniforge3/envs/vaccines/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.

exporting GSE52245.SDY1260, strain no. 3: IgG2_serotype_C, day: D30


[NbConvertApp] Converting notebook export/GSE52245.SDY1260_IgG2_serotype_C_D30_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 532795 bytes to export/GSE52245.SDY1260_IgG2_serotype_C_D30_discard_seroprotected.html


exporting GSE48023.SDY1276, strain no. 2: B_Florida_4_2006, day: AdjMFC


[NbConvertApp] Converting notebook export/GSE48023.SDY1276_B_Florida_4_2006_AdjMFC_discard_seroprotected.ipynb to html
Traceback (most recent call last):
  File "/home/yonatan/miniforge3/envs/vaccines/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3

In [9]:
# Save promising results to a spreadsheet
def generate_html_path(row):
    output_html = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}.html"
    output_html = os.path.join(af.get_dir_by_name('code'), "export", output_html)    
    output_html = '=HYPERLINK("file://' + output_html + '", "Link")'
    return output_html

results = results[cols_to_access] 
results['html_path'] = results.apply(generate_html_path, axis=1)
results.to_csv(f"export/results_{seroprotected_str}{age_restrict_str}{adjFC_str}.csv", index=False, float_format='%.2f')

In [10]:
results

,Dataset,Strain,strain_index,Day,max_difference,"(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)",html_path
78,SDY296,B_Brisbane_60_2008,2,D28.HAI,0.38,0.65,0.34,0.72,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/SDY296_B_Brisbane_60_2008_D28.HAI_discard_seroprotected.html"", ""Link"")"
43,GSE59654.SDY180,A_Brisbane_10_2007,0,AdjMFC,0.36,0.47,0.44,0.80,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59654.SDY180_A_Brisbane_10_2007_AdjMFC_discard_seroprotected.html"", ""Link"")"
44,GSE59654.SDY180,A_Brisbane_10_2007,0,HAI.D28,0.35,0.51,0.48,0.83,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59654.SDY180_A_Brisbane_10_2007_HAI.D28_discard_seroprotected.html"", ""Link"")"
67,GSE59743.SDY400,B_Wisonsin_01_2010,2,AdjMFC,0.32,0.93,0.61,0.90,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59743.SDY400_B_Wisonsin_01_2010_AdjMFC_discard_seroprotected.html"", ""Link"")"
52,GSE59654.SDY404,A_Perth_16_2009,1,HAI.D28,0.30,0.45,0.53,0.83,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59654.SDY404_A_Perth_16_2009_HAI.D28_discard_seroprotected.html"", ""Link"")"
32,GSE52245.SDY1260,IgG_serotype_C,5,D30,0.28,0.59,0.34,0.62,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG_serotype_C_D30_discard_seroprotected.html"", ""Link"")"
29,GSE52245.SDY1260,IgG_serotype_A,4,AdjMFC,0.26,0.32,0.45,0.72,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG_serotype_A_AdjMFC_discard_seroprotected.html"", ""Link"")"
30,GSE52245.SDY1260,IgG_serotype_A,4,D30,0.26,0.38,0.38,0.64,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG_serotype_A_D30_discard_seroprotected.html"", ""Link"")"
7,GSE41080.SDY212,B_Florida_4_2006,2,AdjMFC,0.25,0.76,0.52,0.53,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE41080.SDY212_B_Florida_4_2006_AdjMFC_discard_seroprotected.html"", ""Link"")"
69,SDY1325,Neisseria_meningitidis_strain_A_(F8238),0,AdjMFC,0.25,0.70,0.45,0.61,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/SDY1325_Neisseria_meningitidis_strain_A_(F8238)_AdjMFC_discard_seroprotected.html"", ""Link"")"
